In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
from time import sleep
import torchvision


cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features_covid = torch.load(r'C:\Users\hvard\Desktop\Neeraj_work\Neeraj_work\covid.pt', map_location=torch.device('cpu'))
features_noncovid = torch.load(r'C:\Users\hvard\Desktop\Neeraj_work\Neeraj_work\health.pt',map_location=torch.device('cpu'))

In [4]:
train_labels = torch.zeros((features_covid.shape[0]))
test_labels = torch.ones((features_noncovid.shape[0]))

In [5]:
train_feats = []
train_label = []
for i in range(250,len(test_labels)):
    train_feats.append(features_covid[i])
    train_feats.append(features_noncovid[i])
    train_label.append(train_labels[i])
    train_label.append(test_labels[i])

In [7]:
test_feats = []
test_label = []

In [8]:
for i in range(0,250):
    test_feats.append(features_covid[i])
    test_feats.append(features_noncovid[i])
    test_label.append(train_labels[i])
    test_label.append(test_labels[i])

In [9]:
train_feats = torch.stack(train_feats)
train_label = torch.stack(train_label)
test_feats = torch.stack(test_feats)
test_label = torch.stack(test_label)

In [10]:
## Loading Features and lables
features_train = train_feats
features_tlabel = train_label

In [18]:
features_train.shape

torch.Size([1958, 128])

In [11]:
features_test = test_feats
features_testlabel = test_label

In [12]:
features_testlabel.shape

torch.Size([500])

In [13]:
import pandas as pd

In [14]:
t_np = features_train.squeeze().detach().numpy()
t_np1 = features_test.squeeze().detach().numpy()
t_np2 = features_tlabel.detach().numpy()
t_np3 = features_testlabel.detach().numpy() #convert to Numpy array
df = pd.DataFrame(t_np) 
df1 = pd.DataFrame(t_np1) 
df2 = pd.DataFrame(t_np2) 
df3 = pd.DataFrame(t_np3) #convert to a dataframe
df.to_csv("trainfeatures",index=False) #save to file
df1.to_csv("testfeatures",index=False)
df2.to_csv("trainlabels",index=False)
df3.to_csv("testlabels",index=False)


In [15]:
trainf = pd.read_csv("trainfeatures")
trainl = pd.read_csv("trainlabels")
testf = pd.read_csv("testfeatures")
testl = pd.read_csv("testlabels")

KNearest Neighbors


In [41]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score


X_train, X_test, y_train, y_test = train_test_split(trainf, trainl, test_size=0.3)
k_list = [3, 5, 7, 11, 21, 63, 255]
'''for k in k_list:
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train)
    accuracy = neigh.score(testf, testl)
    print("Accuracy = ",acc," K= ",k)
    '''
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(testf)
cm = confusion_matrix(testl, y_pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print("Accuracy:",accuracy_score(testl, y_pred))
print("Classification Report:\n",classification_report(testl,y_pred))
print("Confusion Matrix:\n",confusion_matrix(testl,y_pred))
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision",precision_score(testl, y_pred))
print("Recall: ", recall_score(testl, y_pred))
print("F1-Score :", f1_score(testl, y_pred))

Accuracy: 0.946
Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.94      0.95       250
         1.0       0.94      0.95      0.95       250

    accuracy                           0.95       500
   macro avg       0.95      0.95      0.95       500
weighted avg       0.95      0.95      0.95       500

Confusion Matrix:
 [[235  15]
 [ 12 238]]
Sensitivity: 0.94
Specificity: 0.952
Precision 0.9407114624505929
Recall:  0.952
F1-Score : 0.9463220675944335


C:\Users\hvard\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Random Forest

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(trainf, trainl, test_size=0.3)
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(testf)
cm = confusion_matrix(testl, y_pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print("Accuracy:",accuracy_score(testl, y_pred))
print("Classification Report:\n",classification_report(testl,y_pred))
print("Confusion Matrix:\n",confusion_matrix(testl,y_pred))
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision",precision_score(testl, y_pred))
print("Recall: ", recall_score(testl, y_pred))
print("F1-Score :", f1_score(testl, y_pred))

C:\Users\hvard\AppData\Local\Temp/ipykernel_17248/1346995333.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,y_train)


Accuracy: 0.948
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.95      0.95       250
         1.0       0.95      0.94      0.95       250

    accuracy                           0.95       500
   macro avg       0.95      0.95      0.95       500
weighted avg       0.95      0.95      0.95       500

Confusion Matrix:
 [[238  12]
 [ 14 236]]
Sensitivity: 0.952
Specificity: 0.944
Precision 0.9516129032258065
Recall:  0.944
F1-Score : 0.9477911646586344


Decision Trees


In [32]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(trainf, trainl, test_size=0.3)
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(testf)
cm = confusion_matrix(testl, y_pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print("Accuracy:",accuracy_score(testl, y_pred))
print("Classification Report:",classification_report(testl,y_pred))
print("Confusion Matrix:\n", confusion_matrix(testl, y_pred))
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision",precision_score(testl, y_pred))
print("Recall", recall_score(testl, y_pred))
print("F1-Score", f1_score(testl, y_pred))

Accuracy: 0.926
Classification Report:               precision    recall  f1-score   support

         0.0       0.92      0.94      0.93       250
         1.0       0.93      0.92      0.93       250

    accuracy                           0.93       500
   macro avg       0.93      0.93      0.93       500
weighted avg       0.93      0.93      0.93       500

Confusion Matrix:
 [[234  16]
 [ 21 229]]
Sensitivity: 0.936
Specificity: 0.916
Precision 0.9346938775510204
Recall 0.916
F1-Score 0.9252525252525253


Naive Bayes


In [33]:
from sklearn.naive_bayes import GaussianNB


X_train, X_test, y_train, y_test = train_test_split(trainf, trainl, test_size=0.3)
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(testf)
cm = confusion_matrix(testl, y_pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print("Accuracy:",accuracy_score(testl, y_pred))
print("Classification Report:",classification_report(testl,y_pred))
print("Confusion Matrix:\n", confusion_matrix(testl, y_pred))
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision",precision_score(testl, y_pred))
print("Recall", recall_score(testl, y_pred))
print("F1-Score", f1_score(testl, y_pred))

Accuracy: 0.94
Classification Report:               precision    recall  f1-score   support

         0.0       0.96      0.92      0.94       250
         1.0       0.92      0.96      0.94       250

    accuracy                           0.94       500
   macro avg       0.94      0.94      0.94       500
weighted avg       0.94      0.94      0.94       500

Confusion Matrix:
 [[230  20]
 [ 10 240]]
Sensitivity: 0.92
Specificity: 0.96
Precision 0.9230769230769231
Recall 0.96
F1-Score 0.9411764705882353


C:\Users\hvard\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVM


In [34]:
from sklearn import svm

clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(testf)
cm = confusion_matrix(testl, y_pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print("Accuracy:",accuracy_score(testl, y_pred))
print("Classification Report:",classification_report(testl,y_pred))
print("Confusion Matrix:\n", confusion_matrix(testl, y_pred))
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision",precision_score(testl, y_pred))
print("Recall", recall_score(testl, y_pred))
print("F1-Score", f1_score(testl, y_pred))

C:\Users\hvard\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Accuracy: 0.946
Classification Report:               precision    recall  f1-score   support

         0.0       0.93      0.96      0.95       250
         1.0       0.96      0.93      0.95       250

    accuracy                           0.95       500
   macro avg       0.95      0.95      0.95       500
weighted avg       0.95      0.95      0.95       500

Confusion Matrix:
 [[240  10]
 [ 17 233]]
Sensitivity: 0.96
Specificity: 0.932
Precision 0.9588477366255144
Recall 0.932
F1-Score 0.9452332657200813


In [35]:
from sklearn import svm

clf = svm.SVC(kernel='poly')
clf.fit(X_train, y_train)
y_pred = clf.predict(testf)
cm = confusion_matrix(testl, y_pred)
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print("Accuracy:",accuracy_score(testl, y_pred))
print("Classification Report:",classification_report(testl,y_pred))
print("Confusion Matrix:\n", confusion_matrix(testl, y_pred))
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision",precision_score(testl, y_pred))
print("Recall", recall_score(testl, y_pred))
print("F1-Score", f1_score(testl, y_pred))

Accuracy: 0.944
Classification Report:               precision    recall  f1-score   support

         0.0       0.93      0.96      0.94       250
         1.0       0.95      0.93      0.94       250

    accuracy                           0.94       500
   macro avg       0.94      0.94      0.94       500
weighted avg       0.94      0.94      0.94       500

Confusion Matrix:
 [[239  11]
 [ 17 233]]
Sensitivity: 0.956
Specificity: 0.932
Precision 0.9549180327868853
Recall 0.932
F1-Score 0.9433198380566802


C:\Users\hvard\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Logistic Regression

In [37]:
from sklearn.linear_model import LogisticRegression


X_train, X_test, y_train, y_test = train_test_split(trainf, trainl, test_size=0.3)
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict_proba(testf)
cm = confusion_matrix(testl, model.predict(testf))
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print("Accuracy:",accuracy_score(testl, model.predict(testf)))
print("Classification Report:\n",classification_report(testl,model.predict(testf)))
print("Confusion Matrix:\n",cm)
print("Precision",precision_score(testl, model.predict(testf)))
print('Sensitivity : ', sensitivity )
print('Specificity : ', specificity)
print("Recall: ", recall_score(testl, model.predict(testf)))
print("f1-score:",f1_score(testl, model.predict(testf)))

Accuracy: 0.962
Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.96      0.96       250
         1.0       0.96      0.97      0.96       250

    accuracy                           0.96       500
   macro avg       0.96      0.96      0.96       500
weighted avg       0.96      0.96      0.96       500

Confusion Matrix:
 [[239  11]
 [  8 242]]
Precision 0.9565217391304348
Sensitivity :  0.956
Specificity :  0.968
Recall:  0.968
f1-score: 0.9622266401590457


C:\Users\hvard\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
